## [MLOps Maturity levels notes](https://github.com/balapriyac/DTC-MLOps-Zoomcamp/blob/main/week1/MLOps-maturity-levels.md)

## Homework 1

In [54]:
import pandas as pd

In [55]:
df_jan = pd.read_parquet('/Users/aasth/Desktop/Data analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/data/yellow_tripdata_2022-01.parquet')
df_feb = pd.read_parquet('/Users/aasth/Desktop/Data analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/data/yellow_tripdata_2022-02.parquet')

### Q1. Read the data for January. How many columns are there?

In [56]:
len(df_jan.columns)

19

### Q.2 What's the standard deviation of the trips duration in January?

In [57]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [58]:
df_jan.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [59]:
df_jan['duration']= (df_jan['tpep_dropoff_datetime']-df_jan['tpep_pickup_datetime']).dt.total_seconds()/60

In [60]:
df_jan['duration'].std()

46.44530513776499

In [61]:
df_jan.shape

(2463931, 20)

### Q3. What fraction of the records left after you dropped the outliers?

In [62]:
df_jan = df_jan[(df_jan['duration']>=1) & (df_jan['duration']<=60)]

In [63]:
df_jan['duration'].describe()

count    2.421440e+06
mean     1.267128e+01
std      8.999282e+00
min      1.000000e+00
25%      6.433333e+00
50%      1.023333e+01
75%      1.613333e+01
max      6.000000e+01
Name: duration, dtype: float64

In [65]:
len(df_jan)/2463931

0.9827547930522406

### Q.4 What's the dimensionality of the feature matrix (number of columns)?

In [68]:
selected_features = ['PULocationID','DOLocationID']
df_jan[selected_features] = df_jan[selected_features].astype(str)

train_dicts = df_jan[selected_features].to_dict(orient='records')

In [70]:
train_dicts[:2]

[{'PULocationID': '142', 'DOLocationID': '236'},
 {'PULocationID': '236', 'DOLocationID': '42'}]

In [71]:
from sklearn.feature_extraction import DictVectorizer

In [73]:
dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)

X_train.shape

(2421440, 515)

### Q.5 What's the RMSE on train?

In [75]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [76]:
lr = LinearRegression()

y_train = df_jan['duration'].values

lr.fit(X_train,y_train)
y_pred = lr.predict(X_train)

print(mean_squared_error(y_train,y_pred,squared=False))

6.986190814952337


### Q.6 What's the RMSE on validation?

In [77]:
df_feb['duration']= (df_feb['tpep_dropoff_datetime']-df_feb['tpep_pickup_datetime']).dt.total_seconds()/60
df_feb = df_feb[(df_feb['duration']>=1) & (df_feb['duration']<=60)]
df_feb[selected_features] = df_feb[selected_features].astype(str)
validation_dicts = df_feb[selected_features].to_dict(orient='records')

X_validation = dv.transform(validation_dicts)
y_pred = lr.predict(X_validation)

print(mean_squared_error(df_feb['duration'].values,y_pred,squared=False))

7.786407163179794
